In [2]:
from parse import parse

In [18]:
NUM_ROUND = 3
inv_tem = 'Invariant Found:	{inv}'
mcmc_tem = "Total MCMC iterations:  {mcmc:d}"
z3_tem = "Total Z3 calls:  {z3:d}"
mcmc_time_tem = "Total MCMC time:  {mcmc_time:f}"
z3_time_tem = "Total Z3 Time:  {z3_time:f}"
thread_tem = "Number of Threads:  {thread:d}"


def read_data(file_path):
    res = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            
            result = parse(inv_tem, line)
            if result is not None:
                res.append({'inv': result['inv']})
                continue     
            result = parse(mcmc_tem, line)
            if result is not None:
                res[-1]['mcmc'] = result['mcmc']
                continue
            result = parse(z3_tem, line)
            if result is not None:
                res[-1]['z3'] = result['z3']
                continue
            result = parse(mcmc_time_tem, line)
            if result is not None:
                res[-1]['mcmc_time'] = result['mcmc_time']
                continue
            result = parse(z3_time_tem, line)
            if result is not None:
                res[-1]['z3_time'] = result['z3_time']
                continue
            result = parse(thread_tem, line)
            if result is not None:
                res[-1]['thread'] = result['thread']
                continue
            
    for i in range(len(res), NUM_ROUND):
        res.append(None)
    
    return res

In [19]:
import os

folder_path = 'output'
name_tem = folder_path + "/{file_name}.txt"
record = {}

for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        result = parse(name_tem, file_path)
        print(result["file_name"])
        data = read_data(file_path)
        record[result["file_name"]] = data        

loop_zilu.benchmark53_polynomial
loop_zilu.benchmark10_conjunctive
loop_zilu.benchmark18_conjunctive
loop_zilu.benchmark40_polynomial
loop_zilu.benchmark14_linear
loop_zilu.benchmark16_conjunctive
loop_zilu.benchmark29_linear
loop_zilu.benchmark02_linear
loop_zilu.benchmark21_disjunctive
loop_zilu.benchmark46_disjunctive
loop_zilu.benchmark48_linear
loop_zilu.benchmark15_conjunctive
loop_zilu.benchmark12_linear
loop_zilu.benchmark49_linear
loop_zilu.benchmark26_linear
loop_zilu.benchmark09_conjunctive
loop_zilu.benchmark43_conjunctive
handcrafted.mock
loop_lit.afnp2014_modified
loop_zilu.benchmark36_conjunctive
loop_zilu.benchmark51_polynomial
loop_zilu.benchmark28_linear
loop_zilu.benchmark08_conjunctive
loop_zilu.benchmark01_conjunctive
loop_zilu.benchmark17_conjunctive
loop_zilu.benchmark38_conjunctive
loop_zilu.benchmark13_conjunctive
loop_zilu.benchmark20_conjunctive
loop_zilu.benchmark37_conjunctive
loop_zilu.benchmark45_disjunctive
loop_zilu.benchmark30_conjunctive
loop_zilu.ben

In [30]:
def trim_dict(record):
    new_record = {}
    for key, value in record.items():
        new_record[key] = {}
        mcmc = None
        z3 = None
        for i in range(NUM_ROUND):
            if value[i] is None:
                continue
            if mcmc is None or value[i]["mcmc"] < mcmc:
                mcmc = value[i]["mcmc"]
                z3 = value[i]["z3"]
        if mcmc is None:
            raise Exception("No data for " + key)
         
        new_record[key]["z3"] = z3
        new_record[key]["mcmc"] = mcmc
    return new_record

def dict_to_latex_table(data):
    table = "\\begin{tabular}{|c|c|c|}\n"
    table += "\\hline\n"
    table += "Benchmark & MCMC Iterations & Z3 Runs \\\\\n"
    table += "\\hline\n"

    for key, value in data.items():
        table += str(key) + " & " + str(value["mcmc"]) + " & " + str(value["z3"]) + "\\\\\n"

    table += "\\hline\n"
    table += "\\end{tabular}"

    return table

latex_table = dict_to_latex_table(trim_dict(record))
print(latex_table)

\begin{tabular}{|c|c|c|}
\hline
Benchmark & MCMC Iterations & Z3 Runs \\
\hline
loop_zilu.benchmark53_polynomial & 4650 & 5\\
loop_zilu.benchmark10_conjunctive & 1029 & 4\\
loop_zilu.benchmark18_conjunctive & 84293 & 9\\
loop_zilu.benchmark40_polynomial & 4722 & 3\\
loop_zilu.benchmark14_linear & 110 & 1\\
loop_zilu.benchmark16_conjunctive & 31182 & 6\\
loop_zilu.benchmark29_linear & 3404 & 13\\
loop_zilu.benchmark02_linear & 590 & 2\\
loop_zilu.benchmark21_disjunctive & 647 & 2\\
loop_zilu.benchmark46_disjunctive & 14091 & 3\\
loop_zilu.benchmark48_linear & 580 & 3\\
loop_zilu.benchmark15_conjunctive & 276512 & 5\\
loop_zilu.benchmark12_linear & 157 & 1\\
loop_zilu.benchmark49_linear & 1374 & 5\\
loop_zilu.benchmark26_linear & 225 & 1\\
loop_zilu.benchmark09_conjunctive & 21 & 1\\
loop_zilu.benchmark43_conjunctive & 145153 & 10\\
handcrafted.mock & 287 & 4\\
loop_lit.afnp2014_modified & 1652 & 8\\
loop_zilu.benchmark36_conjunctive & 8 & 1\\
loop_zilu.benchmark51_polynomial & 5829 & 2\